In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import statsmodels.api as sm
import statsmodels

In [2]:
variable = "essentiality"
col_name = f"Metadata_{variable}"
col_name_in_metadata = "Metadata_dependency_probability"
output_df = pd.DataFrame()

In [3]:
orf_plate_metadata_df = pd.read_csv(
    "output/orf_well_plate_platemap.csv.gz",
    usecols=["Metadata_JCP2022", "Metadata_Platemap"],
)
orf_plate_metadata_df.head()

,Metadata_JCP2022,Metadata_Platemap
0,JCP2022_905588,platemap_1
1,JCP2022_912241,platemap_1
2,JCP2022_900266,platemap_1
3,JCP2022_915129,platemap_1
4,JCP2022_907177,platemap_1


In [4]:
orf_annotation_df = (
    pd.read_csv(
        "../00.download-and-process-annotations/output/orf_metadata.tsv.gz",
        sep="\t",
        usecols=["Metadata_JCP2022", col_name_in_metadata],
    )
    .assign(col=lambda x: np.where(x[col_name_in_metadata] > 0.75, "True", "False"))
    .rename(columns={"col": col_name})
)

orf_df = (
    orf_plate_metadata_df.merge(orf_annotation_df, on="Metadata_JCP2022", how="left")
    .dropna(subset=[col_name])
    .drop_duplicates(subset=["Metadata_JCP2022", "Metadata_Platemap"])
)

orf_df.head()

,Metadata_JCP2022,Metadata_Platemap,Metadata_dependency_probability,Metadata_essentiality
0,JCP2022_905588,platemap_1,0.999821,True
1,JCP2022_912241,platemap_1,0.010326,False
2,JCP2022_900266,platemap_1,0.048173,False
3,JCP2022_915129,platemap_1,NaN,False
4,JCP2022_907177,platemap_1,NaN,False


Fisher's exact test

In [5]:
fisher_df = pd.DataFrame(
    columns=[
        "platemap",
        f"{variable}_False_in_other_platemaps"
        f"{variable}_False_in_platemap",
        f"{variable}_True_in_other_platemaps",
        f"{variable}_True_in_platemap",
        "odds_ratio",
        "pvalue",
        "adjusted_pvalue",
    ]
)

for platemap in orf_df.Metadata_Platemap.unique():
    df = orf_df.assign(platemap_x = lambda x: np.where(x.Metadata_Platemap == platemap, True, False))
    table = sm.stats.Table.from_data(df[["platemap_x", col_name]])

    odds_ratio, pvalue = stats.fisher_exact(table.table)

    output_df = pd.concat(
        [
            output_df,
            pd.DataFrame(
                {
                    "platemap": platemap,
                    f"{variable}_False_in_other_platemaps": int(table.table[0, 0]),
                    f"{variable}_False_in_platemap": int(table.table[0, 1]),
                    f"{variable}_True_in_other_platemaps": int(table.table[1, 0]),
                    f"{variable}_True_in_platemap": int(table.table[1, 1]),
                    "odds_ratio": odds_ratio,
                    "pvalue": pvalue,
                },
                index=[0],
            ),
        ],
        ignore_index=True,
    )

output_df["adjusted_pvalue"] = statsmodels.stats.multitest.fdrcorrection(output_df.pvalue)[1]

In [6]:
print(
    output_df.sort_values(by="adjusted_pvalue")
    .query("adjusted_pvalue < 0.05")
    .to_markdown(index=False)
)

output_df.to_csv(f"output/{variable}_fisher_exact_orf.tsv", sep="\t", index=False)

| platemap    |   essentiality_False_in_other_platemaps |   essentiality_False_in_platemap |   essentiality_True_in_other_platemaps |   essentiality_True_in_platemap |   odds_ratio |      pvalue |   adjusted_pvalue |
|:------------|----------------------------------------:|---------------------------------:|---------------------------------------:|--------------------------------:|-------------:|------------:|------------------:|
| platemap_39 |                                   13656 |                             1478 |                                    333 |                              10 |     0.277463 | 1.74393e-06 |       7.84771e-05 |
| platemap_30 |                                   13660 |                             1475 |                                    329 |                              13 |     0.365937 | 5.79923e-05 |       0.00130483  |
| platemap_22 |                                   13656 |                             1474 |                                    333 